In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import glob
import pickle

# gp_drt_dir = '../comparisons/GP-DRT'
# if gp_drt_dir not in sys.path:
#     sys.path.append(gp_drt_dir)
import GP_DRT as gp
from GP_utils import gp_fit
    
if '../../../bayes-drt' not in sys.path:
    sys.path.append('../../../bayes-drt')
    sys.path.append('../../../../misc_modules')
import eis_utils as gt

%load_ext autoreload
%autoreload 2

reloaded GP_DRT
reloaded GP_utils


In [2]:
def save_pickle(obj, file):
    with open(file,'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print('Dumped pickle to {}'.format(file))
    
def load_pickle(file):
    with open(file,'rb') as f:
        return pickle.load(f)

# Simulated data

In [3]:
# tau for plotting
tau_plot = np.logspace(-7,2,200)
f_plot = 1/(2*np.pi*tau_plot)
start = time.time()
files = glob.glob('../../../data/simulated/Z*.csv')
files = [f for f in files if f.find('DDT')==-1]
error_files = []
theta0s = [[0.1,0.5,1],[0.1,1.5,2],[0.1,5,1]]
for file in files:
    print(file)
    print('-------------------------')
    suffix = file[file.find('_'):-4]
    file_start = time.time()
    best_path = 'results/for_figs'
    Zoutbest = os.path.join(best_path,f'Zout{suffix}.csv')
    Goutbest = os.path.join(best_path,f'Gout{suffix}.csv')
    pklbest = os.path.join(best_path,f'obj_{suffix}.pkl')
    if os.path.exists(Zoutbest) and os.path.exists(Goutbest) and os.path.exists(pklbest):
        print('Already ran')
    else:
    
        best_fun = np.inf
        for n,theta0 in enumerate(theta0s):
            print('theta0=',theta0)
            respath = 'results/theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
            Zoutfile = os.path.join(respath,f'Zout{suffix}.csv')
            Goutfile = os.path.join(respath,f'Gout{suffix}.csv')
            pkl = os.path.join(respath,f'obj_{suffix}.pkl')
    #         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
    #             print('Already ran')
    #         else:
            df = pd.read_csv(file)
            # sort ascending
            df = df.sort_values('Freq')
            Z = df['Zreal'].values + 1j*df['Zimag'].values

            try:
                result = gp_fit(df['Freq'].values,Z,theta0=np.array(theta0),freq_star=f_plot,max_iter=20)

                Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                                     columns=['freq','Zreal','Zimag','sigma_im'])
                g_res = pd.DataFrame(np.array([tau_plot,result['gamma_star'],result['sigma_gamma_star']]).T,
                                     columns=['tau','gamma','sigma_gamma'])
                Z_res.to_csv(Zoutfile,index=False)
                g_res.to_csv(Goutfile,index=False)
                save_pickle(result,pkl)
                # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
                if result['min_result']['fun'] < best_fun:
                    best_fun = result['min_result']['fun']
                    Z_res.to_csv(Zoutbest,index=False)
                    g_res.to_csv(Goutbest,index=False)
                    save_pickle(result,pklbest)
            except np.linalg.LinAlgError:
                error_files.append(file)
                print('LinAlgError')
    
        print('File fit time:', time.time()-file_start)
        
print('Total fit time:',time.time()-start)


../../../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_Macdonald_2.5.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_noiseless.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_Orazem_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_Orazem_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_Orazem_2.5.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_uniform_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_uniform_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_2RC_uniform_2.5.csv
-------------------------
Already ran
../../../data/simulated\Z_2ZARC_Macdonald_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_2ZARC_Macdonald_1.0.csv
-----------------

Dumped pickle to results/theta0=(0.1,5,1)\obj__trunc_uniform_1.0.pkl
File fit time: 254.56210851669312
../../../data/simulated\Z_ZARC-RL_Macdonald_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_Macdonald_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_Macdonald_2.5.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_noiseless.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_Orazem_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_Orazem_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_Orazem_2.5.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_uniform_0.25.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_uniform_1.0.csv
-------------------------
Already ran
../../../data/simulated\Z_ZARC-RL_uniform_2.5.csv
-------------------------
Already ran
../../../data

In [16]:
def errfun():
    raise OverflowError('test')
    
try:
    errfun()
except (np.linalg.LinAlgError,OverflowError) as oe:
    print(oe)
    print('caught')

test
caught


# Experimental data
## LIB

In [9]:
lib_files = ['../../../data/experimental/DRTtools_LIB_data.txt',
             '../../../data/experimental/DRTtools_LIB_data_qtr.csv'
            ]

theta0s = [[1e-4,1e-3,1],[5e-4,1e-3,1],[1e-4,2.5e-3,1],[1e-4,1e-3,2],[1e-4,5e-4,2],[1e-4,5e-4,1],
           [1e-4,5e-4,0.5],[1e-4,1e-3,0.5]]

f_plot = np.logspace(4,-5,200)
tau_plot = 1/(2*np.pi*f_plot)

start = time.time()
error_files = []
for file in lib_files:
    print(file)
    print('-------------------------')
    suffix = file[file.find('_'):-4]
    best_path = 'results/for_figs'
    Zoutbest = os.path.join(best_path,f'Zout{suffix}.csv')
    Goutbest = os.path.join(best_path,f'Gout{suffix}.csv')
    pklbest = os.path.join(best_path,f'obj{suffix}.pkl')
    
#     if os.path.exists(Zoutbest) and os.path.exists(Goutbest) and os.path.exists(pklbest):
#             print('Already ran')
#     else:
    best_fun = np.inf
    file_start = time.time()
    for n,theta0 in enumerate(theta0s):
        print('theta0=',theta0)
        tsuf = suffix + '_theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
        Zoutfile = os.path.join('results/exp',f'Zout{tsuf}.csv')
        Goutfile = os.path.join('results/exp',f'Gout{tsuf}.csv')
        pkl = os.path.join('results/exp',f'obj{tsuf}.pkl')

        if file[-3:]=='txt':
            df = pd.read_csv(file,sep='\t',header=None)
            df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
        else:
            df = pd.read_csv(file)
        # remove inductive tail
        df = df[df['Zimag']<0]
        # sort ascending
        df = df.sort_values('Freq')
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        try:
            result = gp_fit(df['Freq'].values,Z,theta0=theta0,freq_star=f_plot)
            Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                             columns=['freq','Zreal','Zimag','sigma_im'])
            # sort by descending freq
            Z_res = Z_res.sort_values('freq',ascending=False,ignore_index=True)
            g_res = pd.DataFrame(np.array([tau_plot,result['gamma_star'],result['sigma_gamma_star']]).T,
                                 columns=['tau','gamma','sigma_gamma'])

            # save results files
            Z_res.to_csv(Zoutfile,index=False)
            g_res.to_csv(Goutfile,index=False)
            save_pickle(result,pkl)

            # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
            if result['min_result']['fun'] < best_fun:
                best_fun = result['min_result']['fun']
                Z_res.to_csv(Zoutbest,index=False)
                g_res.to_csv(Goutbest,index=False)
                save_pickle(result,pklbest)
        except np.linalg.LinAlgError as lae:
            # cholesky failed for theta0. Can't evaluate NMLL, so no use in fitting
            print(lae)
            print('Can\'t evaluate NMLL for theta0. Skipping')

    print('File fit time: {} min'.format((time.time()-file_start)/60))

print('Total fit time: {} min'.format((time.time()-start)/60))

../../../data/experimental/DRTtools_LIB_data.txt
-------------------------
theta0= [0.0001, 0.001, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0003822	0.0025626	 0.9999998  -736.644
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0001,0.001,1).pkl
Dumped pickle to results/for_figs\obj_LIB_data.pkl
theta0= [0.0005, 0.001, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0010765	0.0040031	 1.0000006  -656.778
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0005,0.001,1).pkl
theta0= [0.0001, 0.0025, 1]
Matrix is not positive definite
Can't evaluate NMLL for theta0. Skipping
theta0= [0.0001, 0.001, 2]
sigma_n,	  sigma_f,	 ell,		NMLL
0.0057036	0.0016470	 1.9999993  -500.709
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0001,0.001,2).pkl
theta0= [0.0001, 0.0005, 2]
sigma_n,	  sigma_f,	

## Protonic ceramic microelectrode

In [5]:
tco_file = '../../../data/experimental/PDAC_COM3_02109_Contact10_2065C_500C.txt'
tco_df = gt.read_eis_zdata(tco_file)

f_plot_tco = np.logspace(7,-3,200)
tau_plot_tco = np.logspace(-7,4,200)

theta0s = [[5e5,1e6,1],[1e6,1e6,1],[1e6,2.5e6,1],[5e5,2.5e6,1],
          [5e5,1e6,2],[5e5,2.5e6,2],[1e6,1e6,2],[5e5,2.5e6,0.5]]

file = tco_file
suffix = '_PDAC'
Zoutbest = os.path.join('results/for_figs',f'Zout{suffix}.csv')
Goutbest = os.path.join('results/for_figs',f'Gout{suffix}.csv')
pklbest = os.path.join('results/for_figs',f'obj{suffix}.csv')

best_fun = np.inf
file_start = time.time()
for n,theta0 in enumerate(theta0s):
    print('theta0=',theta0)
    tsuf = suffix + '_theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
    Zoutfile = os.path.join('results/exp',f'Zout{tsuf}.csv')
    Goutfile = os.path.join('results/exp',f'Gout{tsuf}.csv')
    pkl = os.path.join('results/exp',f'obj{tsuf}.pkl')
    
    df = gt.read_eis_zdata(tco_file)
    # sort ascending
    df = df.sort_values('Freq')
    Z = df['Zreal'].values + 1j*df['Zimag'].values
    
    try:
        result = gp_fit(df['Freq'].values,Z,theta0=theta0,max_iter=10,freq_star=f_plot_tco)

        Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                             columns=['freq','Zreal','Zimag','sigma_im'])
        g_res = pd.DataFrame(np.array([tau_plot_tco,result['gamma_star'],result['sigma_gamma_star']]).T,
                             columns=['tau','gamma','sigma_gamma'])
        # sort by descending freq
        Z_res = Z_res.sort_values('freq',ascending=False,ignore_index=True)

        #save files
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        save_pickle(result,pkl)
        
        # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
        if result['min_result']['fun'] < best_fun:
            best_fun = result['min_result']['fun']
            Z_res.to_csv(Zoutbest,index=False)
            g_res.to_csv(Goutbest,index=False)
            save_pickle(result,pklbest)
    except np.linalg.LinAlgError as lae:
        # cholesky failed for theta0. Can't evaluate NMLL, so no use in fitting
        print(lae)
        print('Can\'t evaluate NMLL for theta0. Skipping')
    
        
print('File fit time:', time.time()-file_start)

theta0= [500000.0, 1000000.0, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
499999.9999773	1000000.0000009	 1.4992727  1415.252
499999.9999121	1000000.0000029	 2.0608005  1414.675
499999.9998102	1000000.0000060	 2.3661658  1414.610
499999.9997546	1000000.0000079	 2.3032102  1414.604
499999.9996921	1000000.0000099	 2.2993189  1414.604
Matrix is not positive definite
Choosing iteration with lowest NMLL: 5
Dumped pickle to results/exp\obj_PDAC_theta0=(500000.0,1000000.0,1).pkl
Dumped pickle to results/for_figs\obj_PDAC.csv
theta0= [1000000.0, 1000000.0, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
999999.9999807	1000000.0000009	 1.5752927  1480.652
999999.9999249	1000000.0000027	 2.3310677  1480.186
999999.9998802	1000000.0000043	 2.2677631  1480.181
999999.9998341	1000000.0000060	 2.2674560  1480.181
999999.9978204	1000000.0000786	 2.2677604  1480.181
999999.9977742	1000000.0000803	 2.2674560  1480.181
999997.8765889	1000000.0765874	 2.2674560  1480.181
999997.8745559	1000000.0766607	 2.2677650  1480.181
999